# 1 引用函数+加载菜单数据 杨苏艳

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import time
import csv
import os
from datetime import datetime
# . 加载菜单数据  存储菜品ID、名称和价格。
def load_menu_data():
    menu_data = {}
    with open('menu.txt', 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3:
                dish_id = parts[0]
                dish_name = parts[1]
                price = float(parts[2])
                menu_data[dish_id] = (dish_name, price)
    return menu_data #使用utf-8编码处理中文菜单 strip()去除首尾空白字符 split()默认按空白字符分割

In [ ]:
# 2. 初始化Spark环境
sc = SparkContext("local[2]", "DishOrderCalculation")
ssc = StreamingContext(sc, 5)  # 5秒批处理间隔 ，每5秒处理一批数据。
# 广播菜单数据
menu_broadcast = sc.broadcast(load_menu_data()) #只读变量缓存到每个节点 避免每个task重复传输
# 全局变量
all_orders = []  # 存储所有订单
last_summary_time = time.time()
output_file = None  # 输出文件名
file_counter = 1    # 文件计数器

In [ ]:
# 生成新的CSV文件名
def generate_new_filename():
    global file_counter
    # 查找已存在的文件数量
    existing_files = [f for f in os.listdir() if f.startswith("收入汇总-") and f.endswith(".csv")]
    file_counter = len(existing_files) + 1
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    return f"收入汇总-{file_counter}_{timestamp}.csv" #用于存储收入汇总结果（按时间戳和计数器命名）

# 2 刘倩 
创建DStream 实时订单数据接收

In [ ]:
# 初始化CSV文件
def init_csv_file():  
    global output_file #全局变量
    output_file = generate_new_filename() #调用 
    with open(output_file, 'w', newline='', encoding='utf-8') as f: #写入模式打开csv文件
        writer = csv.writer(f) #创建csv写入器对象
        writer.writerow(["序号", "菜品ID", "菜名", "价格(元)"])
    print(f"已创建新的收入汇总文件: {output_file}") #打印日志信息

In [ ]:
# 3. 创建DStream 实时订单数据接收
lines = ssc.socketTextStream("localhost", 9999) #创建一个从Socket接收文本数据的输入流
# 4. 处理函数 订单批处理
def process_order_batch(rdd):    #定义批处理函数	每个微批次（5秒）的RDD会触发此函数
    global all_orders, last_summary_time  #声明使用全局变量：all_orders: 存储所有订单、last_summary_time: 记录上次20秒汇总的时间
    if not rdd.isEmpty(): #检查当前批次是否有数据 rdd不为空 就进入if块执行	是为了避免空批次处理，节省资源
        current_time = time.time() #记录当前批次的处理时间
        current_batch = [] #初始化一个空列表，用来临时存储当前批次的订单数据
        # 处理当前批次
        orders = rdd.collect() #将分布式的RDD数据收集到Driver节点
        print("\n=== 新订单明细 ===")
        batch_total = 0.0  #累计当前批次的总金额 实时统计与输出
        for i, dish_id in enumerate(orders, 1): #遍历所有订单并生成序号	从1开始计数
            menu_info = menu_broadcast.value.get(dish_id, ("未知菜品", 0.0))  #从广播变量查询dish.id对应的菜品信息
            dish_name, price = menu_info #解包元组数据，将菜单信息拆分为单独的菜品名称和价格变量
            batch_total += price #累加当前批次金额 计算当前总销售额	用于实时统计 
            current_batch.append((dish_id, dish_name, price)) #保存订单到临时列表current_batch	准备写入CSV
            print(f"{i}. 菜品ID: {dish_id}, 菜名: {dish_name}, 价格: {price}元")  #打印单条订单详情
        
        print(f"当前批次总价: {batch_total}元") #打印当前批次总金额	实时反馈处理结果

        #socketTextStream("host", port)从Socket创建文本数据流（DStream），可设置存储级别（如`MEMORY_AND_DISK`）或合并多数据源（`stream1.union(stream2)
        #rdd.isEmpty()  结合`filter()`预先过滤空数据：`dstream.filter(lambda x: x.strip() != "")
        #rdd.collect()  将分布式RDD数据拉取到Driver端
        #foreachRDD(func)  对每个RDD应用自定义函数（代码中隐含通过`process_order_batch`实现）写入数据库时可用连接池优化：在`foreachPartition`内创建共享连接 
        #broadcast.value.get(key, default)  从广播变量查询菜品信息（无则返回默认值） 广播复杂对象（如模型、配置字典）：`sc.broadcast({"key": "value"})


In [ ]:
        # 先更新全局订单列表
        all_orders.extend(current_batch)
        # 再保存到CSV文件
        save_to_csv(current_batch)
        # 20秒汇总检查
        if current_time - last_summary_time >= 20:
            print_20sec_summary()
            last_summary_time = current_time #更新时间

# 3 喻梦宇

In [ ]:
# 保存订单到CSV
def save_to_csv(batch_orders): #定义一个名为`save_to_csv`的函数，接收一个参数`batch_orders`
    if output_file is None: #检查`output_file`变量是否为None
        init_csv_file()  #如果输出文件未设置，调用`init_csv_file()`函数来初始化CSV文件
    
    with open(output_file, 'a', newline='', encoding='utf-8') as f:  #使用UTF-8编码以追加模式('a')打开输出文件， 使用with语句确保文件操作完成后会自动关闭
        writer = csv.writer(f)  #创建一个CSV写入器对象，用于写入数据到文件
        start_idx = len(all_orders) - len(batch_orders) + 1 #计算起始索引值，基于所有订单的总数(`all_orders`)减去当前批次订单数再加1
        for i, order in enumerate(batch_orders, start=start_idx): #使用enumerate获取索引和订单数据， 索引从之前计算的`start_idx`开始
            writer.writerow([i, order[0], order[1], order[2]])  #将索引i和订单的前三个元素写入CSV文件
    # 更新总收入统计
        update_total_summary()

In [ ]:
# 更新总收入统计
def update_total_summary():
    total = sum(order[2] for order in all_orders)  #计算所有订单中第三列（索引为2）的总和
    with open(output_file, 'a', newline='', encoding='utf-8') as f:  #以追加模式('a')打开一个CSV文件，使用utf-8编码
        writer = csv.writer(f)
        writer.writerow([])  #写入一个空行作为分隔
        writer.writerow(["总计", "", "", f"{total:.1f}"])  # 使用f-string控制格式,写入总计行，包含"总计"和计算出的总金额
        writer.writerow(["总订单数", len(all_orders)])  #写入总订单数，使用`len(all_orders)`获取订单数量
        writer.writerow(["生成时间",datetime.now().strftime("%Y-%m-%d %H:%M:%S")])  #写入当前生成时间，格式为"YYYY-MM-DD HH:MM:SS"

# 4 业务监控需求+实时业务脉搏 贾青逾
##### 快餐店需掌握高峰时段流量（如20秒内突增50单→需临时加派厨师）

In [ ]:
# 打印20秒汇总
def print_20sec_summary():
    last_20sec = all_orders[-20:] if len(all_orders) > 20 else all_orders
    total = sum(order[2] for order in last_20sec)
    print("\n===== 20秒收入汇总 =====")
    print(f"20秒内订单数: {len(last_20sec)}")
    print(f"20秒内总收入: {total}元")
    print("=======================\n")

In [ ]:
# 打印20秒汇总(列表推导式：通过简洁的语法从现有列表生成新列表)
def print_20sec_summary(order):
    # 获取最近20个订单
    current_time = time.time() #time.time()获取时间戳的用法，计算订单是否在20秒内生成
    # (列表推导式：通过简洁的语法从现有列表生成新列表)
    last_20sec = [order for order in all_orders if current_time - order[1] <= 20]
    # all_order全局列表[索引-20：（最后20个元素）]
    # 计算总收入，存入total,sum()对可迭代对象（如列表、生成器）中的数值求和
    total = sum(order[2] for order in last_20sec)
    # 打印汇总信息
    print("\n===== 20秒收入汇总 =====") #标题
    print(f"20秒内订单数: {len(last_20sec)}") #订单数量
    print(f"20秒内总收入: {total}元") #总金额,若需对金额total保留两位小数就round(total，2)
    print("=======================\n")

In [ ]:
# 5. 应用处理函数
lines.foreachRDD(process_order_batch)
# 6. 启动流计算
print("正在启动Spark Streaming程序...")
init_csv_file()  # 程序启动时创建新文件
ssc.start()
ssc.awaitTermination()

In [ ]:
# 6. 启动流计算
print("正在启动Spark Streaming程序...")# 调用预测模型处理订单批次数据。
init_csv_file()# 程序启动时创建新文件初始化 CSV 文件，用于日志或数据存储。
ssc.start()# 启动流式处理服务（sec.start()），基于 Spark 的流计算框架，如 Spark Streaming 。
ssc.awaitTermination()# 终止服务（sec.maintermination()），执行资源清理。
# 方法调用：libcf_forecastRBC()、init_csv_file()、sec.start() 和 sec.maintermination()
# libcf_forecastRBC(process_order_batch)用途：
# 是一个自定义库（libcf）中的函数，用于处理订单批次（process_order_batch）
# 并进行预测（forecast）或资源平衡计算。
# 动态参数调整： 支持不同的预测模型参数libcf_forecastRBC(process_order_batch, model_type="ARIMA", window_size=30)
# 多数据源支持：不仅处理批次数据，还能处理实时流数据
# 回调函数：处理完成后触发回调libcf_forecastRBC(process_order_batch, callback=notify_completion)
# 返回详细结果： 返回预测结果和置信区间result = libcf_forecastRBC(process_order_batch, return_confidence=True)
# init_csv_file()用途：初始化一个 CSV 文件，是用于记录程序运行日志、预测结果或中间数据。
# 其他用法：
# 允许指定文件路径和列名init_csv_file(file_path="output/results.csv", columns=["timestamp", "order_id", "prediction"])
# 追加持追加数据而不是每次覆盖 init_csv_file(mode="append")
# 多文件支持：初始化多个日志文件（如错误日志、性能日志）
# init_csv_file(log_type="error")
# init_csv_file(log_type="performance")
# 3. sec.start()用途：
# 用于启动后台服务、Spark 任务或资源管理器。
# 其他可能的用法： 带参数启动：启动时指定资源配置（如线程数、内存）sec.start(threads=4, memory="8G")
# 异步启动：非阻塞式启动sec.start(async_mode=True)
# 状态检查：启动后返回状态
# status = sec.start()
# if status != "RUNNING":
#     raise RuntimeError("启动失败")
# 4. sec.maintermination()用途：
# sec 组件的清理或终止方法，用于释放资源或关闭服务。
# 其他用法：
# 优雅终止： 支持强制终止或等待任务完成sec.maintermination(force=False)  #等待当前任务完成
# 返回终止状态：返回终止是否成功success = sec.maintermination()
# 回调钩子： 终止前执行自定义清理逻辑

# 代码总览 喻梦宇

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import time
import csv
import os
from datetime import datetime

# 1. 加载菜单数据
def load_menu_data():
    menu_data = {}
    with open('menu.txt', 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3:
                dish_id = parts[0]
                dish_name = parts[1]
                price = float(parts[2])
                menu_data[dish_id] = (dish_name, price)
    return menu_data

# 2. 初始化Spark环境
sc = SparkContext("local[2]", "DishOrderCalculation")
ssc = StreamingContext(sc, 5)  # 5秒批处理间隔

# 广播菜单数据
menu_broadcast = sc.broadcast(load_menu_data())

# 全局变量
all_orders = []  # 存储所有订单
last_summary_time = time.time()
output_file = None  # 输出文件名
file_counter = 1    # 文件计数器

# 生成新的CSV文件名
def generate_new_filename():
    global file_counter
    # 查找已存在的文件数量
    existing_files = [f for f in os.listdir() if f.startswith("收入汇总-") and f.endswith(".csv")]
    file_counter = len(existing_files) + 1
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    return f"收入汇总-{file_counter}_{timestamp}.csv"

# 初始化CSV文件
def init_csv_file():
    global output_file
    output_file = generate_new_filename()
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["序号", "菜品ID", "菜名", "价格(元)"])
    print(f"已创建新的收入汇总文件: {output_file}")

# 3. 创建DStream
lines = ssc.socketTextStream("localhost", 9999)

# 4. 处理函数
def process_order_batch(rdd):
    global all_orders, last_summary_time
    
    if not rdd.isEmpty():
        current_time = time.time()
        current_batch = []
        
        # 处理当前批次
        orders = rdd.collect()
        print("\n=== 新订单明细 ===")
        batch_total = 0.0
        
        for i, dish_id in enumerate(orders, 1):
            menu_info = menu_broadcast.value.get(dish_id, ("未知菜品", 0.0))
            dish_name, price = menu_info
            batch_total += price
            current_batch.append((dish_id, dish_name, price))
            print(f"{i}. 菜品ID: {dish_id}, 菜名: {dish_name}, 价格: {price}元")
        
        print(f"当前批次总价: {batch_total}元")
        
        # 先更新全局订单列表
        all_orders.extend(current_batch)
        # 再保存到CSV文件
        save_to_csv(current_batch)
        # 添加到全局订单列表
        all_orders.extend(current_batch)
        # 20秒汇总检查
        if current_time - last_summary_time >= 20:
            print_20sec_summary()
            last_summary_time = current_time
# 保存订单到CSV
def save_to_csv(batch_orders):
    if output_file is None:
        init_csv_file()
    
    with open(output_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        start_idx = len(all_orders) - len(batch_orders) + 1
        for i, order in enumerate(batch_orders, start=start_idx):
            writer.writerow([i, order[0], order[1], order[2]])
    
    # 更新总收入统计
    update_total_summary()

# 更新总收入统计
def update_total_summary():
    total = sum(order[2] for order in all_orders)
    with open(output_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([])
        writer.writerow(["总计", "", "", f"{total:.1f}"])  # 使用f-string控制格式
        writer.writerow(["总订单数", len(all_orders)])
        writer.writerow(["生成时间", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
# 打印20秒汇总
def print_20sec_summary():
    last_20sec = all_orders[-20:] if len(all_orders) > 20 else all_orders
    total = sum(order[2] for order in last_20sec)
    
    print("\n===== 20秒收入汇总 =====")
    print(f"20秒内订单数: {len(last_20sec)}")
    print(f"20秒内总收入: {total}元")
    print("=======================\n")

# 5. 应用处理函数
lines.foreachRDD(process_order_batch)

# 6. 启动流计算
print("正在启动Spark Streaming程序...")
init_csv_file()  # 程序启动时创建新文件
ssc.start()
ssc.awaitTermination()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/18 05:55:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/root/anaconda3/envs/pyspark/lib/python3.11/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


正在启动Spark Streaming程序...
已创建新的收入汇总文件: 收入汇总-6_20250518-055600.csv


25/05/18 05:56:06 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/18 05:56:06 WARN BlockManager: Block input-0-1747572965800 replicated to only 0 peer(s) instead of 1 peers
25/05/18 05:56:06 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/18 05:56:06 WARN BlockManager: Block input-0-1747572966200 replicated to only 0 peer(s) instead of 1 peers
25/05/18 05:56:06 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/18 05:56:06 WARN BlockManager: Block input-0-1747572966400 replicated to only 0 peer(s) instead of 1 peers
25/05/18 05:56:06 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/18 05:56:06 WARN BlockManager: Block input-0-1747572966600 replicated to only 0 peer(s) instead of 1 peers
25/05/18 05:56:07 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/18 05:56:07 WARN BlockManager: Block input-0-1747572966800 replicated to


=== 新订单明细 ===
1. 菜品ID: 1, 菜名: 香菇肥牛, 价格: 58.0元
2. 菜品ID: 2, 菜名: 麻婆豆腐, 价格: 32.0元
3. 菜品ID: 3, 菜名: 红烧茄子, 价格: 15.0元
4. 菜品ID: 4, 菜名: 小炒凉粉, 价格: 16.0元
5. 菜品ID: 5, 菜名: 京酱肉丝, 价格: 22.0元
6. 菜品ID: 6, 菜名: 辣椒鱼头, 价格: 48.0元
7. 菜品ID: 1, 菜名: 香菇肥牛, 价格: 58.0元
8. 菜品ID: 2, 菜名: 麻婆豆腐, 价格: 32.0元
9. 菜品ID: 3, 菜名: 红烧茄子, 价格: 15.0元
10. 菜品ID: 4, 菜名: 小炒凉粉, 价格: 16.0元
当前批次总价: 312.0元
